In [ ]:
import iDEA
import numpy as np

In [ ]:
def softcore_interaction(x)
    n_gridpoints = len(x)
    v_int = np.zeros((n_gridpoints, n_gridpoints))
    for i in range(n_gridpoints):
        for j in range(n_gridpoints):
            v_int[i, j] = 1/np.sqrt((x[i]-x[j])**2+1)
    return v_int

In [ ]:
def make_data(potentials,x):
  n_gridpoints = len(x)
  n_potentials = len(potentials)
  v_int = softcore_interaction(x)
  densities_1e = np.zeros(shape=(n_potentials,n_gridpoints))
  densities_2e = np.zeros(shape=(n_potentials,n_gridpoints))
  densities_3e = np.zeros(shape=(n_potentials,n_gridpoints))
  energies_1e = np.zeros(shape=(n_potentials,1))
  energies_2e = np.zeros(shape=(n_potentials,1))
  energies_3e = np.zeros(shape=(n_potentials,1))
  for i in range(n_potentials):
    print(f"Potential {i+1} out of {n_potentials}")
    print("Solving for 1 electron")
    electrons_1e = 'u'
    system_1e = iDEA.system.System(x,potentials[i,:],v_int,electrons_1e)
    state_1e = iDEA.methods.interacting.solve(system_1e,k=0)
    energies_1e[i]= state_1e.energy
    densities_1e[i,:]=iDEA.observables.density(system_1e,state_1e)

    print("Solving for 2 electrons")
    electrons_2e = 'ud'
    system_2e = iDEA.system.System(x,potentials[i,:],v_int,electrons_2e)
    state_2e = iDEA.methods.interacting.solve(system_2e,k=0)
    energies_2e[i]= state_2e.energy
    densities_2e[i,:]=iDEA.observables.density(system_2e,state_2e)

    print("Solving for 3 electrons")
    electrons_3e = 'udd'
    system_3e = iDEA.system.System(x,potentials[i,:],v_int,electrons_3e)
    state_3e = iDEA.methods.interacting.solve(system_3e,k=0)
    energies_3e[i]= state_3e.energy
    densities_3e[i,:]=iDEA.observables.density(system_3e,state_3e)

  ionisations_1e = -energies_1e
  ionisations_2e = energies_1e-energies_2e
  ionisations_3e = energies_2e-energies_3e
  affinities_1e = ionisations_2e
  affinities_2e = ionisations_3e
  gaps_1e = ionisations_1e - affinities_1e
  gaps_2e = ionisations_2e - affinities_2e
  universals_1e = energies_1e-np.trapz(potentials*densities_1e,x,axis=1).reshape(energies_1e.shape)
  universals_2e = energies_2e-np.trapz(potentials*densities_2e,x,axis=1).reshape(energies_2e.shape)
  universals_3e = energies_3e-np.trapz(potentials*densities_3e,x,axis=1).reshape(energies_3e.shape)
  return densities_1e,densities_2e,densities_3e,energies_1e,energies_2e,energies_3e,ionisations_2e,ionisations_3e,affinities_1e,affinities_2e,universals_1e,universals_2e,universals_3e,gaps_1e,gaps_2e

In [ ]:
def atomic_potential(x,Z,alpha):
  return -Z/np.sqrt(x**2+alpha**2)

In [ ]:
#defining the atomic potentials
Zs=np.linspace(3,4.1,12) #effective nuclear charge values
betas=np.linspace(1.2,1.75,12)
alphas 1/betas #softening parameter values
n_tot = len(Zs)*len(betas)
x = np.linspace(-6,6,65) #spatial grid
potentials = np.zeros(shape=(n_tot,x.size))
i=0
for Z in Zs:
  for alpha in alphas:
    potentials[i,:] = atomic_potential(x,Z,alpha)
    i+=1

In [ ]:
#calculating all electronic properties
densities_1e,densities_2e,densities_3e,energies_1e,energies_2e,energies_3e,ionisations_2e,ionisations_3e,affinities_1e,affinities_2e,universals_1e,universals_2e,universals_3e,gaps_1e,gaps_2e = make_data(potentials,x)

In [ ]:
np.savez('full_dataset.npz',potentials=potentials,densities_1e=densities_1e,densities_2e=densities_2e,densities_3e=densities_3e,
         energies_1e=energies_1e,energies_2e=energies_2e,energies_3e=energies_3e,ionisations_2e=ionisations_2e,ionisations_3e=ionisations_3e,
         affinities_1e=affinities_1e,affinities_2e=affinities_2e,universals_1e=universals_1e,universals_2e=universals_2e,universals_3e=universals_3e,
         gaps_1e=gaps_1e,gaps_2e=gaps_2e)